# 1. Import

In [17]:
# import tensorflow as tf
# import numpy as np
# import matplotlib.pyplot as plt
# import os 
# from PIL import Image
# import glob
from tensorflow import keras

# 2. 데이터 불러오기 + Resize

In [7]:
# 데이터 확인
file_path = '1.jpg'
with Image.open(file_path) as img:
    width, height = img.size
print(f"가로: {width} px, 세로: {height} px")

가로: 28 px, 세로: 28 px


In [5]:
pwd

'/Users/kenny/데싸4기/Practice/Rock_Scissor_Paper'

In [6]:
cd scissor

/Users/kenny/데싸4기/Practice/Rock_Scissor_Paper/scissor


In [12]:
def resize_images(img_path):
    images = glob.glob(os.path.join(img_path, "*.jpg"))
    print(len(images), "images to be resized.")

    target_size = (28, 28)
    for img_file in images:
        with Image.open(img_file) as old_img:
            # Image.ANTIALIAS 대신 Image.LANCZOS 사용
            new_img = old_img.resize(target_size, Image.LANCZOS)
            new_img.save(img_file, "JPEG")

    print(len(images), "images resized.")

# HOME 경로 아래 실제 폴더를 정확히 지정
home = os.getenv("HOME")  # e.g. "/Users/kenny"
image_dir_path = os.path.join(home, "데싸4기", "Practice", "Rock_Scissor_Paper", "scissor")

resize_images(image_dir_path)
print("가위 이미지 resize 완료!")
        


100 images to be resized.
100 images resized.
가위 이미지 resize 완료!


In [13]:
image_dir_path = os.path.join(home, "데싸4기", "Practice", "Rock_Scissor_Paper", "paper")

resize_images(image_dir_path)
print("보 이미지 resize 완료!")
        

100 images to be resized.
100 images resized.
보 이미지 resize 완료!


In [14]:
image_dir_path = os.path.join(home, "데싸4기", "Practice", "Rock_Scissor_Paper", "rock")

resize_images(image_dir_path)
print("바위 이미지 resize 완료!")
        

100 images to be resized.
100 images resized.
바위 이미지 resize 완료!


# 3. 라벨링

In [23]:
def load_data(img_path, number_of_data=420):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/Scissors-samples/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/Rocks-samples/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/Paper-samples/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/데싸4기/Practice/Rock_Scissor_Paper/test"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 420 입니다.
x_train shape: (420, 28, 28, 3)
y_train shape: (420,)


# 4. 딥러닝 설계

In [18]:
# 아래의 하이퍼파라미터는 예시입니다. 높은 인식률을 얻기 위해 하이퍼파라미터를 수정해 보세요. 
n_channel_1=16
n_channel_2=32
n_dense=32
n_train_epoch=30

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()


/Users/kenny/miniforge3/envs/rsp_dl/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-07 01:28:03.717337: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-06-07 01:28:03.717468: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-06-07 01:28:03.717486: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-06-07 01:28:03.717830: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-07 01:28:03.718236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,819 (120.39 KB)

 Trainable params: 30,819 (120.39 KB)

 Non-trainable params: 0 (0.00 B)

# 5. 딥러닝 학습

In [19]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train, y_train, epochs=n_train_epoch)

Epoch 1/30


2025-06-07 01:28:37.932363: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.3523 - loss: 54.9168
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4301 - loss: 12.3446
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6698 - loss: 2.6219
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7686 - loss: 1.4987
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6773 - loss: 1.6367
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7492 - loss: 1.3938
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8446 - loss: 0.8629
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9111 - loss: 0.4069
Epoch 9/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9004 - loss: 0.3438
Epoch 10/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9643 - loss: 0.1169
Epoch 11/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9846 - loss: 0.0369
Epoch 12/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0

# 6. 테스트

In [25]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.

image_dir_path = os.getenv("HOME") + "/데싸4기/Practice/Rock_Scissor_Paper/test/Scissors-samples"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/데싸4기/Practice/Rock_Scissor_Paper/test/Rocks-samples"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/데싸4기/Practice/Rock_Scissor_Paper/test/Paper-samples"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/데싸4기/Practice/Rock_Scissor_Paper/test"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화
print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

127 images to be resized.
127 images resized.
158 images to be resized.
158 images resized.
135 images to be resized.
135 images resized.
학습데이터(x_train)의 이미지 개수는 420 입니다.
x_test shape: (420, 28, 28, 3)
y_test shape: (420,)


In [26]:

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

14/14 - 1s - 54ms/step - accuracy: 0.4095 - loss: 8.6132
test_loss: 8.613218307495117 
test_accuracy: 0.4095238149166107


# 결론 및 시사점
	•	정확도 40%
	•	“100개 테스트 이미지 중 약 40개”만 맞춘 상태이며,
	•	무작위(33%)보다는 조금 나은 정도지만, 실용적인 수준에서는 매우 낮은 편입니다.
	•	loss 8.6
	•	테스트 이미지에 대해 모델이 “정답일 확률”을 거의 0으로 판단하면서 완전히 자신 있게 틀린 예측을 내놓은 샘플이 많다는 의미입니다.
	•	손실 함수가 −log(정답 확률)이므로, 정답 확률이 0.01 정도면 손실값이 약 4.6, 0.001이면 약 6.9, 0.0001이면 약 9.2가 됩니다.
	•	따라서 “여러 샘플에 대해 정답 확률이 0.0001 근처였다” 같은 결과가 평균되어 최종 loss가 8.6으로 나온 것입니다.

⸻

개선을 위해 시도해 볼 수 있는 것들
	1.	데이터 전처리/증강(Data Augmentation)
	•	학습 단계에서 이미지 회전, 반전, 밝기 조정 등을 활용해 데이터 다양성을 늘리면,
	•	테스트 이미지가 약간 달라도 더 견고하게 예측할 수 있습니다.
	2.	모델 구조 수정
	•	현재 사용하신 CNN 구조가 너무 얕거나, 필터 개수가 부족해서 “특징 학습”이 잘 되지 않을 수도 있습니다.
	•	레이어를 더 쌓거나, 필터 개수를 늘려 보세요.
	3.	하이퍼파라미터 튜닝
	•	학습률(learning rate), 배치 크기(batch size), epoch 수 등을 조절하여 “훈련 성능”과 “일반화 성능” 사이의 균형을 찾아야 합니다.
	4.	학습–테스트 데이터 정합성 확인
	•	테스트 이미지가 원래 학습 데이터와 비슷하게 취합된 것인지(예: 같은 배경, 같은 해상도) 확인하고,
	•	필요하다면 학습 데이터를 늘리거나 더 다양한 예시를 포함해서 모델이 “낯선” 테스트 패턴을 잘 받아들이도록 하는 것이 중요합니다.